In [1]:
root_dir = "/data/Full_datasets"
base_dir = root_dir + 'ulmfit-lab/'

In [2]:
!pip install pythainlp[thai2fit]

In [3]:
!pip install emoji

In [4]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import pythainlp

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

##Load data

First, we load the data from disk into a Dataframe.
A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.
**Note: the data is provided courtesy of [TrueVoice](http://www.truevoice.co.th/) and must be used for educational purposes only**

In [5]:
data_df = pd.read_csv('full_datasets.csv')
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()


,text,intent
0,เกาะติดเศรษฐกิจการเงินน้ำมันแพงเกี่ยวข้องกับค่า,น้ำมัน
1,มิยคนไทยซดน้ำมันแพงวเศษรับไม่ต่ำกว่า30บลิตร,น้ำมัน
2,ต่างชาติถล่มขายหุ้นเครือปตทมาร์เก็ตแคปพคหายแสน...,น้ำมัน
3,คาดโอเปคไม่ผลิตเพิ่มหวั่นน้ำมันราคาพุ่งยันปริม...,น้ำมัน
4,สศกคาดผลไม้ปี49ล้านตลาดพิษน้ำมันส่งผลกำลังซื้อ...,น้ำมัน


,text,intent
count,991587,991587
unique,987350,9
top,เวิลด์แก๊สเคียงข้างสร้างรอยยิ้ม,ธนาคาร
freq,6,267349


### Split data

split data into train and test set 

In [6]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_df,stratify=data_df['intent'], test_size=0.2,random_state=42)


In [7]:
train_data.head()

,text,intent
561233,พลวัตเศรษฐกิจปัญหาเศรษฐกิจการเงินโลกจาก,ธนาคาร
762893,บสยเข้าเครดิตบูโรดึงsmeเข้าระบบ,ธนาคาร
607633,กองทุนเน้นลงทุนหุ้นแบงก์พลังงานเชื่อมั่นเศรษฐก...,ธนาคาร
473798,พาณิชย์ปิ๊งขายไข่ไก่ร้านถูกใจ,ธุรกิจค้าปลีก
112747,โดรนปริศนาโจมตีคลังน้ำมันซาอุฯ,น้ำมัน


## [ULMFit](https://arxiv.org/abs/1801.06146) Model
ULMFit model has 3 stages:
1. The LM is trained on a general-domain corpus to capture
general features of the language in different layers
2. The full LM is fine-tuned on target task data 
3. The classifier is fine-tuned on the target task using gradual unfreezing, ‘Discr’, and STLR to
preserve low-level representations and adapt high-level ones 

In [8]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

from fastai.text import * 
from fastai.callbacks import CSVLogger, SaveModelCallback

from pythainlp.ulmfit import *

model_path = ''

In [9]:
_THWIKI_LSTM #The LM is trained on a general-domain corpus to capture general features of the language in different layers

{'wgts_fname': '/root/pythainlp-data/thwiki_model_lstm.pth',
 'itos_fname': '/root/pythainlp-data/itos_lstm.pkl'}

In [10]:
#configuration
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=None, max_vocab=60000, min_freq=3)]

data_lm = (TextList.from_df(train_data, model_path, cols=['text'], processor=processor)
    .random_split_by_pct(valid_pct = 0.01, seed = 1412)
    .label_for_lm()
    .databunch(bs=64))
data_lm.sanity_check()
data_lm.save('truevoice_cu_lm.pkl')

/root/miniconda3/lib/python3.7/site-packages/fastai/data_block.py:217: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


In [11]:
data_lm = load_data(model_path,'truevoice_cu_lm.pkl') #Target task data
data_lm.sanity_check()
len(data_lm.train_ds), len(data_lm.valid_ds)

(785337, 7932)

In [12]:
data_lm.show_batch(5)

idx,text
0,กรอบ xxbos พิษ ระเบิด จ่อ เลื่อน ไทยแลนด์ โฟกัส xxbos โครงสร้าง การค้า โลก เปลี่ยน เหตุ ส่งออก เอเชีย ฟื้นตัว ช้า xxbos ฟันธง ผลกระทบ คดี ปตท นักวิเคราะห์ มอง ความเสียหาย มี มากกว่า ที่ xxbos ชาย คาธ ก สระ วัง ข่าวลือ ข่าว ลวง หลอก จ่าย เงิน xxbos พระราชทาน 10 ล้าน ช่วย ผู้ประสบภัย เนปาล 64 นัก แสวงบุญ ถึง บ้าน เล่า xxbos สุริยะ อัด แคมเปญ
1,ด้วย ของกำนัล xxbos makro ลือ ขาย กิจการ ไม่ เลิก โบ รก ฯ คาด ราคาขาย 730 บาท xxbos สธ เฝ้า ระวัง 483 นักเดินทาง พท เสี่ยง อี โบ ลา xxbos มท 3 ขู่ ยึด 4 สนามกอล์ฟ ระยอง xxbos อาเซียน ระทึก จับ 8 เจ ไอ ซ่องสุม ภูเก็ต พังงา พบ ระเบิด จ้อง ป่วน xxbos เฮดจ์ฟันด์ เท ขาย สัญญา ล่วงหน้า ทุก สินค้า ยาง รับ กระทบ
2,หุ้น แลกเปลี่ยน ความคิดเห็น xxbos launch ร้อน จาก แบงก์ ขยาย วงเงิน ช่วย เหยื่อ น้ำท่วม ปี xxbos ลาว สร้าง สนามบิน ใหม่ ที่ อัตตะ ปือ xxbos ลูกขุน ชี้ ซิม ป์ สัน ผิด จริง ไม่รอด คุก แน่ xxbos ทางออก นอกตำรา ลุง ตู่ ปฏิวัติ ระบบ กู้ยืม ไชโย ต้น xxbos สำนักข่าว ดาวเหนือ แหล่ง ช้อป ท็อป 8 ใน ลอน ดอน xxbos เวิลด์ แบงก์ ประเมิน ศก
3,จับจ่าย xxbos ปฏิทิน ธุรกิจ วันที่ 20 กันยายน 2561 xxbos ฟินัน ซ่า ทิ้ง หุ้น ไทย ซบ เวียดนาม จีน ชู ผลตอบแทน 20 xxbos น้ำลด 7 แสน คน ส่อ ตกงาน แนะ รัฐ อัด งบ หนัก ดูแล ค้าปลีก หนุน กระตุ้น แรง xxbos ชีพจร โลก ธุรกิจ ศุ ภา ลัย ผุด คอนโด หรู ส่งท้าย ปี xxbos ผล โพลล์ ยัน โลก มอง อเมริกา เป็น ภัย คุกคาม xxbos
4,โดนใจ ไม่ แบ่ง สี xxbos ปตท ตั้งเป้า รับ ฤดูกาล 2013 หนุน พลัง เพลิง ขึ้น ไทย พ รี เมีย xxbos ifs รุก สินเชื่อ เอสเอ็มอี วาง เป้าหมาย ปี นี้ โต 10 xxbos รวบ 5 ต้องสงสัย ยึด รพ ทหารพราน ปิดล้อม 5 จุด ระ แงะ นราฯ ป่วน อีก บึ้ม xxbos เดินหน้า เลี้ยวซ้าย น้ำมัน แพง ต้อง ฉลาด เติม xxbos อี โบ ลา คืนชีพ ตาย 2


### (step 1) load pretrained model trained on general corpus

In [13]:
#configuration
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args) #for step 2

# load pretrained models ( Step 1. The LM is trained on a general-domain corpus to capture
# general features of the language in different layers)
learn.load_pretrained(**_THWIKI_LSTM) 

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (785337 items)
x: LMTextList
xxbos   พลวัต เศรษฐกิจ ปัญหา เศรษฐกิจ การเงิน โลก จาก,xxbos   บสย เข้า เครดิต บู โร ดึง sme เข้า ระบบ,xxbos   กองทุน เน้น ลงทุน หุ้น แบงก์ พลังงาน เชื่อมั่น เศรษฐกิจ ไทย คึกคัก,xxbos   พาณิชย์ ปิ๊ง ขาย ไข่ไก่ ร้าน ถูกใจ,xxbos   โด รน ปริศนา โจมตี คลังน้ำมัน ซาอุฯ
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (7932 items)
x: LMTextList
xxbos   ทิ้ง หมัด เข้ามุม คำถาม ซ้ำซาก สาม จังหวัด ชายแดน ใต้,xxbos   cpw จ่อ ไอ พีโอ 160 ล้าน หุ้น ต่อ ยอด สาขา สินค้า ดิจิทัล,xxbos   ลูก คนดัง ชวน ประดิษฐ์ การ์ด diy บอ กรัก แม่,xxbos   สหกรณ์ จังหวัด แพร่ ตรวจ เยี่ยม แปลง ผลิต,xxbos   สื่อ นอก พูด คำเตือน ที่ ต้อง รับฟัง
y: LMLabelList
,,,,
Path: .;

Test: None, model=SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(30008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(30008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400,

### (step 2) the full LM is fine-tuned on target task data 

Read about [freeze_to](https://docs.fast.ai/basic_train.html#Learner.freeze_to)

In [14]:
#train frozen
print('training frozen')
learn.freeze_to(-1)
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

training frozen


epoch,train_loss,valid_loss,accuracy,time
0,5.764125,5.513730,0.240753,06:58


[unfreeze](https://docs.fast.ai/basic_train.html#Learner.unfreeze) sets every layer group to trainable 

In [15]:
#train unfrozen (take about 5 mins)
print('training unfrozen')
learn.unfreeze()
learn.fit_one_cycle(10, 1e-4, moms=(0.8, 0.7))

training unfrozen


epoch,train_loss,valid_loss,accuracy,time
0,5.484977,5.275995,0.259619,08:33
1,5.249506,5.061747,0.275989,08:33
2,5.082715,4.890694,0.289509,08:34
3,4.962525,4.780730,0.298108,08:33
4,4.869998,4.709342,0.303380,08:33
5,4.811227,4.661977,0.307196,08:33
6,4.793129,4.632424,0.309758,08:33
7,4.781185,4.617828,0.311033,08:34
8,4.751219,4.610410,0.311501,08:34
9,4.754484,4.609965,0.311628,08:33


In [16]:
learn.save_encoder('truevoice_enc')

### (step 3) Classification
 The classifier is fine-tuned on the target task using gradual unfreezing, ‘Discr’, and STLR to
preserve low-level representations and adapt high-level ones 

In [20]:
#configuration
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=data_lm.vocab, max_vocab=60000, min_freq=3)]


data_cls = (TextList.from_df(train_data, model_path, cols=['text'], processor=processor)
    .random_split_by_pct(valid_pct = 0.05, seed = 1412)
    .label_from_df('intent')
    .add_test(TextList.from_df(test_data, model_path, cols=['text'], processor=processor))
    .databunch(bs=64)
    )

data_cls.sanity_check()
data_cls.save('truevoice_cu_cls.pkl')

/root/miniconda3/lib/python3.7/site-packages/fastai/data_block.py:217: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


In [22]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(bptt=70, drop_mult=0.5, alpha=2, beta=1)

learn = text_classifier_learner(data_cls, AWD_LSTM, config=config, pretrained=False, **trn_args)
learn.opt_func = partial(optim.Adam, betas=(0.7, 0.99))

#load pretrained finetuned model
learn.load_encoder('truevoice_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (753606 items)
x: TextList
xxbos   พลวัต เศรษฐกิจ ปัญหา เศรษฐกิจ การเงิน โลก จาก,xxbos   บสย เข้า เครดิต บู โร ดึง sme เข้า ระบบ,xxbos   พาณิชย์ ปิ๊ง ขาย ไข่ไก่ ร้าน ถูกใจ,xxbos   โด รน ปริศนา โจมตี คลังน้ำมัน ซาอุฯ,xxbos   สำรวจ การออม ของ แรงงาน เผย เงิน หลัง เกษียณ ไม่ พอใช้
y: CategoryList
ธนาคาร,ธนาคาร,ธุรกิจค้าปลีก,น้ำมัน,ธนาคาร
Path: .;

Valid: LabelList (39663 items)
x: TextList
xxbos   ทิ้ง หมัด เข้ามุม คำถาม ซ้ำซาก สาม จังหวัด ชายแดน ใต้,xxbos   cpw จ่อ ไอ พีโอ 160 ล้าน หุ้น ต่อ ยอด สาขา สินค้า ดิจิทัล,xxbos   ลูก คนดัง ชวน ประดิษฐ์ การ์ด diy บอ กรัก แม่,xxbos   สหกรณ์ จังหวัด แพร่ ตรวจ เยี่ยม แปลง ผลิต,xxbos   สื่อ นอก พูด คำเตือน ที่ ต้อง รับฟัง
y: CategoryList
สงคราม,ธุรกิจค้าปลีก,ธนาคาร,ธุรกิจค้าปลีก,ธนาคาร
Path: .;

Test: LabelList (198318 items)
x: TextList
xxbos   จับ ประเด็น ชี้ แหล่ง ก๊าซ อ่าวไทย หยุด ไม่ กระทบ ผลิต ไฟฟ้า,xxbos   ชุมทาง ตลาด เดอะ สกาย ช็อปปิ้ง พร้อม เปิด พย นี้,xxbos   ข่าวสั้น เศรษฐกิจ ยกระดับ อู่

In [23]:
#train unfrozen
learn.freeze_to(-1)
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.743507,0.661197,0.786148,04:37


In [24]:
#gradual unfreezing (5 mins)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3 / (2.6 ** 4), 5e-3), moms=(0.8, 0.7))
learn.unfreeze()
learn.fit_one_cycle(4, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7), 
                   callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='truevoice_cls')])

epoch,train_loss,valid_loss,accuracy,time
0,0.549097,0.515926,0.834556,05:16


epoch,train_loss,valid_loss,accuracy,time
0,0.510950,0.470091,0.849784,07:08


epoch,train_loss,valid_loss,accuracy,time
0,0.451193,0.462586,0.853062,10:55
1,0.424389,0.457966,0.857096,10:46
2,0.330062,0.482420,0.858634,11:03
3,0.253293,0.522244,0.855634,11:01


Better model found at epoch 0 with accuracy value: 0.8530620336532593.
Better model found at epoch 1 with accuracy value: 0.8570960164070129.
Better model found at epoch 2 with accuracy value: 0.8586339950561523.


# Evaluation

In [25]:
probs, _ = learn.get_preds(ds_type = DatasetType.Test, ordered=True)
classes = learn.data.train_ds.classes
preds = np.array([classes[i] for i in probs.argmax(1).numpy()])
prob = probs.numpy()

In [26]:
print("ULMFit Model Acc: %f%%" % ((test_data['intent']== preds).sum() / test_data.shape[0] * 100))
print("ULMFit Model Micro f1: %f%%" % f1_score(test_data['intent'], preds, average='micro') )
print("ULMFit Model Macro f1: %f%%" % f1_score(test_data['intent'], preds, average='macro') )

ULMFit Model Acc: 85.553001%
ULMFit Model Micro f1: 0.855530%
ULMFit Model Macro f1: 0.853765%


In [27]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(test_data['intent'], preds, target_names=classes))

               precision    recall  f1-score   support

       การบิน       0.86      0.82      0.84     17156
       ธนาคาร       0.86      0.90      0.88     53470
ธุรกิจค้าปลีก       0.81      0.81      0.81     19984
       น้ำมัน       0.84      0.80      0.82     23160
 พลังงานไฟฟ้า       0.83      0.73      0.78     12106
    ภัยพิบัติ       0.86      0.89      0.87     11765
       สงคราม       0.90      0.93      0.91     20486
       อสังหา       0.86      0.85      0.85     34977
     โรคระบาด       0.92      0.92      0.92      5214

     accuracy                           0.86    198318
    macro avg       0.86      0.85      0.85    198318
 weighted avg       0.86      0.86      0.85    198318

